# CPE342 - Karena Task1

In [4]:
import pandas as pd

In [5]:
train_df = pd.read_csv("Dataset/task1/train.csv")
test_df = pd.read_csv("Dataset/task1/test.csv")

In [6]:
train_df.head()

,id,player_id,kill_death_ratio,headshot_percentage,win_rate,accuracy_score,kill_consistency,reaction_time_ms,account_age_days,level,...,team_play_score,buy_decision_score,map_knowledge,clutch_success_rate,first_blood_rate,survival_time_avg,damage_per_round,utility_usage_rate,crosshair_placement,is_cheater
0,PLY00001,P058048,4.556072,15.494589,NaN,36.953785,0.737653,NaN,1204.586641,73.364603,...,0.653589,0.860407,0.487273,0.534021,0.321734,NaN,91.517681,0.736361,0.928522,0.0
1,PLY00002,P037400,6.177695,51.476210,86.670187,59.946064,NaN,93.748524,3.405453,84.104334,...,NaN,0.410925,0.690833,0.151872,0.462196,84.599562,119.212421,0.525739,0.946558,0.0
2,PLY00003,P001054,6.584511,51.692578,32.827778,46.545739,0.811803,120.895839,551.221686,43.803175,...,0.639732,0.619968,0.511507,0.311445,0.270611,83.793333,58.200726,0.739374,0.897872,0.0
3,PLY00004,P062710,5.237014,44.181842,62.740475,30.260066,0.345491,248.056545,114.034430,NaN,...,0.766887,0.612241,0.477374,NaN,0.402723,97.518653,121.315491,0.531533,0.860302,1.0
4,PLY00005,P068548,2.453744,44.361101,68.296046,27.470153,0.851962,210.530688,153.210944,83.293728,...,0.691935,0.653540,0.698389,NaN,0.197220,50.405967,79.110571,0.771739,0.853758,1.0


In [7]:
test_df.head()

,id,player_id,kill_death_ratio,headshot_percentage,win_rate,accuracy_score,kill_consistency,reaction_time_ms,account_age_days,level,...,communication_rate,team_play_score,buy_decision_score,map_knowledge,clutch_success_rate,first_blood_rate,survival_time_avg,damage_per_round,utility_usage_rate,crosshair_placement
0,ANS00001,P128393,4.807460,NaN,85.513445,40.885421,0.541166,206.216513,554.404426,26.449873,...,0.466226,0.407562,0.574111,NaN,0.239997,0.331286,39.167105,133.600242,0.854674,0.810149
1,ANS00002,P057262,4.478905,54.549192,66.998239,41.730924,0.866650,80.200970,718.385124,30.350951,...,0.850628,0.825845,0.717767,0.604164,0.258227,0.218852,84.855443,67.842696,0.408059,NaN
2,ANS00003,P063924,6.982952,37.740360,76.489177,46.397954,NaN,141.426624,525.868237,69.546770,...,0.444088,0.908123,0.538952,0.618745,0.407880,0.261211,98.825249,65.697213,0.285722,0.822645
3,ANS00004,P051190,5.003208,44.231043,69.877125,33.585034,0.959418,110.800568,1089.867093,41.780396,...,0.601710,0.445652,0.464034,0.865429,0.139233,0.127039,31.225586,139.140240,0.626042,0.959571
4,ANS00005,P023176,3.038168,NaN,NaN,29.467070,0.567550,84.594256,109.235604,83.159919,...,0.300572,0.534116,NaN,0.758842,0.346107,0.471993,NaN,48.957231,0.774780,0.872152


In [8]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import fbeta_score, make_scorer, classification_report

1. Define the F2 Scorer

In [9]:
f2_scorer = make_scorer(fbeta_score, beta=2)

In [10]:
target = 'is_cheater'

if train_df[target].isnull().any():
    print(f"Found {train_df[target].isnull().sum()} rows with missing target. Removing them.")
    train_df = train_df.dropna(subset=[target])
else:
    print("No missing target values found. Good to go.")

Found 2124 rows with missing target. Removing them.


In [11]:
target = 'is_cheater'
features = [col for col in train_df.columns if col not in ['id', 'player_id', 'is_cheater']]

In [12]:
X_train = train_df[features]
y_train = train_df[target]
X_test = test_df[features]

In [13]:
X_train.head()

,kill_death_ratio,headshot_percentage,win_rate,accuracy_score,kill_consistency,reaction_time_ms,account_age_days,level,level_progression_speed,friend_network_size,...,communication_rate,team_play_score,buy_decision_score,map_knowledge,clutch_success_rate,first_blood_rate,survival_time_avg,damage_per_round,utility_usage_rate,crosshair_placement
0,4.556072,15.494589,NaN,36.953785,0.737653,NaN,1204.586641,73.364603,0.033039,122.0,...,0.620589,0.653589,0.860407,0.487273,0.534021,0.321734,NaN,91.517681,0.736361,0.928522
1,6.177695,51.476210,86.670187,59.946064,NaN,93.748524,3.405453,84.104334,12.211503,0.0,...,0.851946,NaN,0.410925,0.690833,0.151872,0.462196,84.599562,119.212421,0.525739,0.946558
2,6.584511,51.692578,32.827778,46.545739,0.811803,120.895839,551.221686,43.803175,0.076185,205.0,...,0.636306,0.639732,0.619968,0.511507,0.311445,0.270611,83.793333,58.200726,0.739374,0.897872
3,5.237014,44.181842,62.740475,30.260066,0.345491,248.056545,114.034430,NaN,NaN,75.0,...,0.799637,0.766887,0.612241,0.477374,NaN,0.402723,97.518653,121.315491,0.531533,0.860302
4,2.453744,44.361101,68.296046,27.470153,0.851962,210.530688,153.210944,83.293728,0.354636,79.0,...,0.242936,0.691935,0.653540,0.698389,NaN,0.197220,50.405967,79.110571,0.771739,0.853758


In [14]:
y_train.head()

0    0.0
1    0.0
2    0.0
3    1.0
4    1.0
Name: is_cheater, dtype: float64

In [15]:
test_ids = test_df['id']

In [16]:
print(f"Training with {len(features)} features.")
print(f"Training data shape: {X_train.shape}")
print(f"Training data shape: {y_train.shape}")
print(f"Test data shape: {X_test.shape}")

Training with 31 features.
Training data shape: (97748, 31)
Training data shape: (97748,)
Test data shape: (25889, 31)


In [17]:
if y_train.sum() == 0:
    print("Warning: No 'cheater' (1) samples found in the training data.")
    scale_pos_weight = 1.0
else:
    scale_pos_weight = (y_train == 0).sum() / y_train.sum()

print(f"Class Imbalance: Legitimate (0) to Cheater (1) ratio: {scale_pos_weight:.2f}")
print(f"Using 'scale_pos_weight' = {scale_pos_weight:.2f} for the model.")

Class Imbalance: Legitimate (0) to Cheater (1) ratio: 1.86
Using 'scale_pos_weight' = 1.86 for the model.


In [18]:
print("Setting up the machine learning pipeline...")

pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('model', lgb.LGBMClassifier(
        random_state=42,
        scale_pos_weight=scale_pos_weight,  # Apply the imbalance weight
        objective='binary',
        n_jobs=-1
    ))
])

Setting up the machine learning pipeline...


In [19]:
param_grid = {
    'model__n_estimators': [100, 200, 400],
    'model__learning_rate': [0.05, 0.1],
    'model__num_leaves': [31, 50]
}

In [20]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("Starting model tuning (GridSearchCV)...")
print("This may take a few minutes.")

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring=f2_scorer,  # Optimize for our F2-score
    cv=cv,
    n_jobs=-1,
    verbose=2
)

Starting model tuning (GridSearchCV)...
This may take a few minutes.


In [21]:
grid_search.fit(X_train, y_train)

print("\nModel tuning complete.")

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[LightGBM] [Info] Number of positive: 34129, number of negative: 63619
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005967 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7438
[LightGBM] [Info] Number of data points in the train set: 97748, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.349153 -> initscore=-0.622765
[LightGBM] [Info] Start training from score -0.622765

Model tuning complete.


In [22]:
print(f"Best F2-score from tuning: {grid_search.best_score_:.4f}")
print(f"Best parameters found: {grid_search.best_params_}")

Best F2-score from tuning: 0.7949
Best parameters found: {'model__learning_rate': 0.05, 'model__n_estimators': 100, 'model__num_leaves': 31}


In [23]:
best_model = grid_search.best_estimator_

In [32]:
import joblib
print(f"Saving the best model to 'anti_cheat_model.joblib'...")
model_filename = 'Model/anti_cheat_model_V1'
joblib.dump(best_model, model_filename)
print(f"Model saved successfully to {model_filename}.")

Saving the best model to 'anti_cheat_model.joblib'...
Model saved successfully to Model/anti_cheat_model_V1.


In [24]:
try:
    importances = best_model.named_steps['model'].feature_importances_
    
    feature_importance_df = pd.DataFrame({
        'feature': features,
        'importance': importances
    }).sort_values(by='importance', ascending=False)

    print("\nTop 15 Most Important Features:")
    print(feature_importance_df.head(15))
except Exception as e:
    print(f"\nCould not retrieve feature importances: {e}")

# --- 10. Make Predictions on Test Data ---
print("\nMaking predictions on the test dataset...")
test1_predictions = best_model.predict(X_test)




Top 15 Most Important Features:
                    feature  importance
9       friend_network_size         686
6          account_age_days         672
10         reports_received         354
30      crosshair_placement         233
20         game_sense_score         168
7                     level         167
1       headshot_percentage         120
8   level_progression_speed         108
0          kill_death_ratio         105
3            accuracy_score          79
19      spray_control_score          72
18        aiming_smoothness          54
2                  win_rate          52
4          kill_consistency          52
5          reaction_time_ms          16

Making predictions on the test dataset...


C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [25]:
test1_predictions

array([1., 0., 1., ..., 1., 0., 0.], shape=(25889,))